## Building a Retrieval-Augmented Generation (RAG) System with LangChain

### Introduction

In this notebook, we will learn how to build a Retrieval-Augmented Generation (RAG) system using LangChain in Python. RAG systems combine information retrieval and natural language generation to produce answers that are grounded in external knowledge bases. This approach is particularly useful when dealing with large documents or datasets where direct querying isn’t efficient or possible.

### Objectives

- Understand the concept of Retrieval-Augmented Generation (RAG).
- Learn how to use LangChain to implement a RAG system.
- Implement the system step by step with guided TODO tasks.
- Test your implementation at each step.
- Provide helpful explanations and definitions.

Help

### Methods Used:

- LangChain: A library for building language model applications.
- VectorStore (FAISS): A tool for efficient similarity search and clustering of dense vectors.
- OpenAI Embeddings: Representations of text that can capture semantic meaning.
- RetrievalQA Chain: Combines retrieval and question-answering over documents.

### Data Used

- I extracted some chapters of the Gen AI course as a txt file. 
- The goal how this notebook is to build a RAG system that can answer questions based on the content of these chapters.

## Step 1: Set Up Your Environment

We need to import the required modules and set up the OpenAI API key.

In [6]:
# Import necessary libraries
import sys
from dotenv import load_dotenv
from langchain import hub
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.documents.base import Document
from langchain_core.prompts import ChatPromptTemplate
from typing import List

In [7]:
load_dotenv()
sys.path.append("../")

## Step 2: Load and Split Documents

Load the document you want to use and split it into manageable chunks.

In [8]:
# TODO: Load your document and split it into chunks
# Hint: Use TextLoader and RecursiveCharacterTextSplitter

filename = r"C:\Users\Aycha\Desktop\M2_BDIA\GENAI\GenAI-Dauphine-Course\data\gen_ai_course.txt"
# Answer:
loader = TextLoader(filename, encoding='utf-8')
documents = loader.load()

# Answer
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)


## Step 3: Create Embeddings and Build the VectorStore

Generate embeddings for each chunk and store them in a vector store for efficient retrieval.

In [9]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")


In [10]:
# TODO: Create embeddings and store them in a VectorStore
# Hint: Use OpenAIEmbeddings and FAISS
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectorstore = FAISS.from_documents(documents=docs, embedding=embeddings)
# Hint : Use GoogleGenerativeAIEmbeddings(model=...)

## Step 4: Set Up the QA Chain using LCEL 

Create a chain that can retrieve relevant chunks and generate answers based on them.

In [15]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
# Initialize ChatGoogleGenerativeAI with the required arguments

#Create a function to format documents for the prompt
def format_docs(docs: List[Document]):
    # Hint: Join the content of each document
    return "\n".join(doc.page_content for doc in docs) # Join the page content of docs into a stringg

# Hint: Define the prompt template with system and human messages. See help below

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that answers questions based on the provided documents."),
    ("human", "Here are some relevant documents to answer the question."),
    ("human", "{documents}"),
    ("human", "User's question: {question}"),
    ("assistant", "Answer:"),
])
# Hint: Format the documents using the function above
formatted_docs = format_docs(docs) 

# Hint: Create the QA chain by combining the prompt and model
qa_chain = LLMChain(prompt=prompt, llm=llm)


## Step 5: Ask Questions and Get Answers

Test the system by asking a question.

In [16]:
# TODO: Ask a question to the QA chain
# Replace 'Your question here' with an actual question and run the qa_chain for this question

# Answer:
query = "What is the main topic discussed in the document?"
result = qa_chain.run(documents=formatted_docs, question=query)
print(result)


The document primarily discusses Generative AI with Large Language Models (LLMs). It covers a wide range of topics related to LLMs, including:

* **Building LLMs:** This includes pretraining, tokenization, evaluating, data preprocessing, scaling laws, training processes, costs, and optimization.  It also covers fine-tuning techniques like Supervised Fine Tuning (SFT), Reinforcement Learning from Human Feedback (RLHF), reward models, and Proximal Policy Optimization (PPO) and Direct Preference Optimization (DPO).
* **Transformers:** The document details the architecture of transformers, explaining key concepts such as self-attention, cross-attention, multi-head attention, residual connections, layer normalization, feed-forward layers, softmax layers, and positional embeddings. It also touches upon pre-transformer architectures like N-grams, RNNs, and LSTMs.
* **Retrieval Augmented Generation (RAG):** This section explains how RAG combines retrieval-based and generation-based models, its

## Step 6: Test Your Implementation with Different Questions

Try out different questions to see how the system performs.

In [17]:
# Replace 'Another question here' with your own question and run the qa_chain for this question

query = "Can you summarize the key points mentioned?"
result = qa_chain.run(documents=formatted_docs, question=query)
print(result)

The document covers several key aspects of Generative AI and Large Language Models (LLMs):

**Building LLMs:**  It explains the process of pre-training LLMs, including introduction, cross-entropy loss, tokenization (using Byte-Pair Encoding), evaluation (using perplexity and datasets), data preprocessing, scaling laws, training process, cost, and optimizations.  It also details fine-tuning techniques like Supervised Fine-Tuning (SFT), Reinforcement Learning from Human Feedback (RLHF) with reward models, Proximal Policy Optimization (PPO) and Direct Preference Optimization (DPO), along with their evaluation and challenges.  Data preprocessing is highlighted as crucial for quality.  Scaling laws help determine model and data size based on compute resources.  SFT is the first step in post-training, and RLHF/DPO align the model with human preferences.  Chain-of-Thought (COT) prompting improves efficiency.

**Transformers Architecture:** The document describes the evolution of language mode

## Step 7: Improve the System

You can experiment with different parameters, like adjusting the chunk size or using a different language model.

On peut essayer avec ca llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

Conclusion

Congratulations! You’ve built a simple Retrieval-Augmented Generation system using LangChain. This system can retrieve relevant information from documents and generate answers to user queries.

Help

- TextLoader: Loads text data from files.
- RecursiveCharacterTextSplitter: Splits text into smaller chunks for better processing.
- FAISS: A library for efficient similarity search of embeddings.
- RetrievalQA Chain: A chain that retrieves relevant documents and answers questions based on them.
- OpenAIEmbeddings: Generates embeddings that capture the semantic meaning of text.

## Help

In [14]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

prompt_value = template.invoke(
    {
        "name": "Bob",
        "user_input": "What is your name?"
    }
)

# Output:
# ChatPromptValue(
#    messages=[
#        SystemMessage(content='You are a helpful AI bot. Your name is Bob.'),
#        HumanMessage(content='Hello, how are you doing?'),
#        AIMessage(content="I'm doing well, thanks!"),
#        HumanMessage(content='What is your name?')
#    ]
#)